<a href="https://colab.research.google.com/github/woneuy01/AudioToText/blob/master/Reddit_title_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime

In [0]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [0]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [307]:
from datetime import datetime

### input time here
time_before="20/06/01 00:00:00" # to if set as 5/9 end date input 5/10
time_before = datetime.strptime(time_before, '%y/%m/%d %H:%M:%S')
time_after="20/05/29 00:00:00" # from
time_after = datetime.strptime(time_after, '%y/%m/%d %H:%M:%S')
#now = datetime.now()

before = datetime.timestamp(time_before)
after = datetime.timestamp(time_after)
before = str(int(before)) #October 1st
after = str(int(after)) #January 1st 
print("timestamp =",  after,before )

timestamp = 1590710400 1590969600


In [0]:
#Subreddit to query
sub='coronavirus'
query = ""
subCount = 0
subStats = {}


In [309]:
import datetime
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    try:
      data = getPushshiftData(query, after, before, sub)
    except Exception:
      pass
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590710400&before=1590969600&subreddit=coronavirus
100
2020-05-29 02:51:24
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590720684&before=1590969600&subreddit=coronavirus
100
2020-05-29 05:04:50
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590728690&before=1590969600&subreddit=coronavirus
100
2020-05-29 10:39:24
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590748764&before=1590969600&subreddit=coronavirus
100
2020-05-29 14:07:08
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590761228&before=1590969600&subreddit=coronavirus
100
2020-05-29 16:27:28
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590769648&before=1590969600&subreddit=coronavirus
100
2020-05-29 18:57:43
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590778663&before=1590969600&subreddit=

In [310]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

1794 submissions have added to list
1st entry is:
HOW HIGH WIND SPEED CAN REDUCE NEGATIVE EFFECTS OF CONFIRMED CASES AND TOTAL DEATHS OF COVID-19 INFECTION IN SOCIETY created: 2020-05-29 00:03:45
Last entry is:
New coronavirus losing potency, top Italian doctor says created: 2020-05-31 23:59:29


In [0]:
# def updateSubs_file():
#     upload_count = 0
#     location = ""
#     print("input filename of submission file, please add .csv")
    #filename = input()
    #file = location + filename
    # with open(file, 'w', newline='', encoding='utf-8') as file: 
    #     a = csv.writer(file, delimiter=',')
    #     headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
    #     a.writerow(headers)
    #     for sub in subStats:
    #         a.writerow(subStats[sub][0])
    #         upload_count+=1
               
        #print(str(upload_count) + " submissions have been uploaded")


In [311]:
posts=[]
for sub in subStats:
  posts.append(subStats[sub][0]) 
 
df=pd.DataFrame(posts, columns=["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"])
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,gshz6t,HOW HIGH WIND SPEED CAN REDUCE NEGATIVE EFFECT...,https://poseidon01.ssrn.com/delivery.php?ID=14...,D-R-AZ,1,2020-05-29 00:03:45,4,/r/Coronavirus/comments/gshz6t/how_high_wind_s...,World
1,gsi2rf,Effects of Climatological Parameters on the Ou...,https://europepmc.org/article/ppr/ppr165634,D-R-AZ,1,2020-05-29 00:09:34,2,/r/Coronavirus/comments/gsi2rf/effects_of_clim...,Europe
2,gsi6fu,Spain will not stop use of HCQ to treat COVID-...,https://www.wionews.com/world/spain-will-not-s...,SafePangolin3,1,2020-05-29 00:15:23,0,/r/Coronavirus/comments/gsi6fu/spain_will_not_...,Europe
3,gsi6ta,White House and CDC remove coronavirus warning...,https://www.washingtonpost.com/health/white-ho...,DrAnthonyFauci,1,2020-05-29 00:16:04,62,/r/Coronavirus/comments/gsi6ta/white_house_and...,USA
4,gsi71y,Pennsylvania Democrats Say They Weren't Told W...,https://www.npr.org/sections/coronavirus-live-...,semaforic,1,2020-05-29 00:16:28,2,/r/Coronavirus/comments/gsi71y/pennsylvania_de...,USA


In [312]:
df.sort_values("Total No. of Comments", axis = 0, ascending = False, 
                 inplace = True, na_position ='last') 
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
1331,gtnc3z,U.S. cities fear protests may fuel new wave of...,https://www.cp24.com/world/u-s-cities-fear-pro...,OneBernie2020Please,1,2020-05-30 22:31:59,4194,/r/Coronavirus/comments/gtnc3z/us_cities_fear_...,USA
366,gsspt5,Monkeys 'escape with COVID-19 samples' after a...,https://news.sky.com/story/coronavirus-monkeys...,clonetheory,1,2020-05-29 13:09:16,2266,/r/Coronavirus/comments/gsspt5/monkeys_escape_...,World
1702,gu4czd,"Italy Is Reopening for Tourism in June, But No...",https://www.thrillist.com/news/nation/italy-re...,angjiankai02,1284,2020-05-31 18:39:28,1639,/r/Coronavirus/comments/gu4czd/italy_is_reopen...,World
868,gt648s,"Colorado, USA: COVID-19 Survivor Receives Firs...",https://www.thedenverchannel.com/news/local-ne...,Tactless2U,1,2020-05-30 02:07:59,1515,/r/Coronavirus/comments/gt648s/colorado_usa_co...,USA
5,gsi7li,People are accidentally throwing out their sti...,https://www.nbcnews.com/business/consumer/peop...,txiao007,1,2020-05-29 00:17:28,1264,/r/Coronavirus/comments/gsi7li/people_are_acci...,USA


In [313]:
df=df[df["Total No. of Comments"]>300]
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
1331,gtnc3z,U.S. cities fear protests may fuel new wave of...,https://www.cp24.com/world/u-s-cities-fear-pro...,OneBernie2020Please,1,2020-05-30 22:31:59,4194,/r/Coronavirus/comments/gtnc3z/us_cities_fear_...,USA
366,gsspt5,Monkeys 'escape with COVID-19 samples' after a...,https://news.sky.com/story/coronavirus-monkeys...,clonetheory,1,2020-05-29 13:09:16,2266,/r/Coronavirus/comments/gsspt5/monkeys_escape_...,World
1702,gu4czd,"Italy Is Reopening for Tourism in June, But No...",https://www.thrillist.com/news/nation/italy-re...,angjiankai02,1284,2020-05-31 18:39:28,1639,/r/Coronavirus/comments/gu4czd/italy_is_reopen...,World
868,gt648s,"Colorado, USA: COVID-19 Survivor Receives Firs...",https://www.thedenverchannel.com/news/local-ne...,Tactless2U,1,2020-05-30 02:07:59,1515,/r/Coronavirus/comments/gt648s/colorado_usa_co...,USA
5,gsi7li,People are accidentally throwing out their sti...,https://www.nbcnews.com/business/consumer/peop...,txiao007,1,2020-05-29 00:17:28,1264,/r/Coronavirus/comments/gsi7li/people_are_acci...,USA


In [0]:
#produce excel
#caution encoding="uft-8" does not work. only "utf-8-sig"
#df.to_csv(str(after)+".csv",encoding="utf-8-sig")

In [0]:
#df.title text file no index&header
df["Title"].to_csv(str(time_after)+".txt",index=False,header=None, encoding="utf-8-sig")